## Course: Data Science for Linguists
## Winter semester 23/24
## Assignment: Project (part 1)
## Student: Alla Savinkina

For movie subtitles dataset, I used https://www.opensubtitles.org  and saved the subtitles by genre folder. I also saved the genre information about a movie provided into a txt file. The extracted subtitles and the provided info is saved at subtitles_raw


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: 

In [ ]:
cd /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/

/content/gdrive/MyDrive/rus_phraseology/subtitles_raw


In [ ]:
!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (85.9 kB/s)
Selecting previously unselected package tree.
(Reading database ... 124565 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
! tree -d

.
├── action
│   ├── Aleksandr. Nevskaya bitva
│   ├── avgust.vosmogo.(2012).rus.1cd.(4502841)
│   ├── brat.2.(2000).rus.1cd.(8790607)
│   ├── brother.(1997).rus.1cd.(8787692)
│   ├── day.watch.(2006).rus.1cd.(5046171)
│   ├── goryachie.novosti
│   ├── kandagar.(2010).rus.1cd.(3647464)
│   ├── My_Iz_Budushchego_2
│   ├── nulevoy.kilometr.(2007).rus.1cd.(8490987)
│   ├── put.(2009).rus.1cd.(3537799)
│   ├── skalolazka.i.posledniy.iz.sedmoy.kolybeli.(2007).rus.1cd.(9507516)
│   ├── svoi.(2004).rus.1cd.(8147497)
│   ├── Taras_Bulba
│   ├── The.Last.Frontier.
│   ├── Tiski 
│   ├── Turetskii_gambit_Rus_2005.Srt
│   └── v.avguste.44go.(2001).rus.1cd.(5527162)
├── adventure
│   ├── 1812.ulanskaya.ballada.(2012).rus.1cd.(8609652)
│   ├── baba.yaga.spasaet.mir.(2023).rus.1cd.(10101161)
│   ├── gagarin.pervyy.v.kosmose.(2013).rus.1cd.(8666869)
│   ├── geograf.globus.propil.(2013).rus.1cd.(9952387)
│   ├── geroy.nashego.vremeni.(2006).rus.1cd.(8862069)
│   ├── koktebel.(2003).rus.1cd.(12777195)


Unfortunately, the provided genre information typically has several entries for each movie and it is impossible to specify exactly one, so I keep the genres as it is provided on opensubtitles website.

### 1. Preprocess srt files

In [ ]:
import os
import re
import chardet
import codecs
import shutil

In [ ]:
def detect_and_convert(file_path):
    """
    Detect file encoding and convert it to UTF-8.
    """
    try:
        with open(file_path, 'rb') as f:
            rawdata = f.read()
        result = chardet.detect(rawdata)
        encoding = result['encoding'] or 'utf-8-sig'
        print(f"Detected encoding: {encoding} for {file_path}")
        return rawdata.decode(encoding, 'ignore')
    except Exception as e:
        print(f"Error detecting or converting file {file_path}: {e}")
        return None

def process_srt_content(content):
    """
    Preprocess the SRT content to remove XML tags, timestamps, line numbers, and unwanted patterns.
    """
    def remove_unwanted_patterns(line):
        line = re.sub(r'\{\d+\}\{\d+\}-?', '', line)  # Remove {1446}{1614}-
        line = line.replace('|', ' ')  # Remove "|"
        return line.strip()

    def remove_xml_tags(text):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)

    processed_content = []
    current_sentence = ""

    for line in content.splitlines():
        line = line.strip()
        line = remove_xml_tags(line)
        if not line or line.isdigit() or '--> ' in line:
            continue
        cleaned_line = remove_unwanted_patterns(line)
        current_sentence += " " + cleaned_line if current_sentence else cleaned_line

        if re.search(r'[.!?]+$', cleaned_line):
            processed_content.append(current_sentence.strip())
            current_sentence = ""

    if current_sentence:
        processed_content.append(current_sentence.strip())

    return "\n".join(processed_content)

def process_and_save_subtitles_flat(input_root, output_folder):
    """
    Process SRT files from the input directory and save all processed files into a single output folder.
    """
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

    for root, _, files in os.walk(input_root):
        for file in files:
            if not file.endswith('.srt'):  # Skip non-SRT files
                continue

            # Construct full input file path
            input_file_path = os.path.join(root, file)

            # Detect and convert encoding
            raw_content = detect_and_convert(input_file_path)
            if raw_content is None:
                print(f"Skipping file due to encoding issue: {input_file_path}")
                continue

            # Process the content
            processed_content = process_srt_content(raw_content)

            # Define output file path in the flat folder
            output_file_path = os.path.join(output_folder, file)

            # Save the processed content
            with open(output_file_path, 'w', encoding='utf-8') as f:
                f.write(processed_content)

            print(f"Processed and saved: {output_file_path}")


In [ ]:
input_directory = '/content/gdrive/MyDrive/rus_phraseology/subtitles_raw'
output_directory = '/content/gdrive/MyDrive/rus_phraseology/subtitles_processed/srt'
process_and_save_subtitles_flat(input_directory, output_directory)

Detected encoding: UTF-8-SIG for /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/mira.(2022).rus.1cd.(9472402)/mira.srt
Processed and saved: /content/gdrive/MyDrive/rus_phraseology/subtitles_processed/srt/mira.srt
Detected encoding: UTF-8-SIG for /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/baba.yaga.spasaet.mir.(2023).rus.1cd.(10101161)/baba_yaga_spasaet_mir.srt
Processed and saved: /content/gdrive/MyDrive/rus_phraseology/subtitles_processed/srt/baba_yaga_spasaet_mir.srt
Detected encoding: utf-8 for /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/1812.ulanskaya.ballada.(2012).rus.1cd.(8609652)/1812_ulanskaya_ballada.srt
Processed and saved: /content/gdrive/MyDrive/rus_phraseology/subtitles_processed/srt/1812_ulanskaya_ballada.srt
Detected encoding: utf-8 for /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/sokrovishcha.partizanskogo.lesa.(2023).rus.1cd.(12591284)/sokrovishcha_partizanskogo.srt
Processed and saved: /cont

In [ ]:
def copy_txt_files(input_path, output_path):
    """
    Copies all `.txt` files from the input directory (including subdirectories)
    to the specified output directory.
    """
    os.makedirs(output_path, exist_ok=True)  # Ensure the output folder exists

    for root, _, files in os.walk(input_path):
        for file in files:
            if file.endswith('.txt'):  # Check if the file is a `.txt` file
                source_file_path = os.path.join(root, file)
                target_file_path = os.path.join(output_path, file)
                try:
                    shutil.copy2(source_file_path, target_file_path)
                    print(f"Copied: {source_file_path} to {target_file_path}")
                except Exception as e:
                    print(f"Failed to copy {source_file_path}: {e}")

In [ ]:
# copy txt files with the genres info
input_directory = '/content/gdrive/MyDrive/rus_phraseology/subtitles_raw'
output_directory = '/content/gdrive/MyDrive/rus_phraseology/subtitles_processed/txt'

copy_txt_files(input_directory, output_directory)

Copied: /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/mira.(2022).rus.1cd.(9472402)/mira.txt to /content/gdrive/MyDrive/rus_phraseology/subtitles_processed/txt/mira.txt
Copied: /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/baba.yaga.spasaet.mir.(2023).rus.1cd.(10101161)/baba_yaga_spasaet_mir.txt to /content/gdrive/MyDrive/rus_phraseology/subtitles_processed/txt/baba_yaga_spasaet_mir.txt
Copied: /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/1812.ulanskaya.ballada.(2012).rus.1cd.(8609652)/1812_ulanskaya_ballada.txt to /content/gdrive/MyDrive/rus_phraseology/subtitles_processed/txt/1812_ulanskaya_ballada.txt
Copied: /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/sokrovishcha.partizanskogo.lesa.(2023).rus.1cd.(12591284)/sokrovishcha_partizanskogo.txt to /content/gdrive/MyDrive/rus_phraseology/subtitles_processed/txt/sokrovishcha_partizanskogo.txt
Copied: /content/gdrive/MyDrive/rus_phraseology/subtitles_raw/adventure/up

#### 2. Extract genres info from .txt files

In [ ]:
import os
import csv
import re
import pandas as pd

In [ ]:
def extract_movie_info_from_txt(folder_path, output_csv, append=False):
    """
    Extracts movie information from .txt files in a folder and saves it to a CSV file.
    """
    header = ['filename', 'title', 'genre']
    write_mode = 'a' if append else 'w'
    write_header = not append or not os.path.exists(output_csv)

    with open(output_csv, write_mode, encoding='utf-8', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        if write_header:
            writer.writeheader()

        for file_name in os.listdir(folder_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(folder_path, file_name)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()
                        title = None
                        genre = []
                        genre_start = False

                        for i, line in enumerate(lines):
                            line = line.rstrip()
                            if not line.strip():  # Skip empty lines
                                continue

                            if title is None:
                                title = line.strip()
                                if "Russian subtitles" in title:
                                    title = title[:title.index("Russian subtitles")].strip()
                                title = re.sub(r'["\']', '', title).strip()
                                title = title.lower()

                            if line.lower().strip() == 'genre:':
                                genre_start = True
                                continue

                            if genre_start:
                                if line.startswith(' '):  # Genre lines start with a space
                                    genre.append(line.strip().lower())
                                else:
                                    genre_start = False

                        print(f"Processing file: {file_name}")
                        print(f"Extracted title: {title}")
                        print(f"Extracted genres: {genre}")

                        writer.writerow({
                            'filename': file_name[:-4],  # Remove .txt from the filename
                            'title': title or 'Unknown',
                            'genre': ', '.join(genre) if genre else 'Unknown'
                        })

                except Exception as e:
                    print(f"Failed to process {file_name}: {e}")

def sort_csv_alphabetically(csv_file):
    """
    Sorts a CSV file alphabetically by the 'title' column.
    """
    try:
        # Load the CSV into a DataFrame
        df = pd.read_csv(csv_file)

        # Sort alphabetically by 'title'
        df = df.sort_values(by='title')

        # Write the sorted DataFrame back to the CSV
        df.to_csv(csv_file, index=False)
        print(f"CSV sorted alphabetically by title and saved: {csv_file}")
    except Exception as e:
        print(f"Failed to sort CSV file {csv_file}: {e}")


In [ ]:
# extract the movie genres info from txt and save to .csv
txt_folder_path = '/content/gdrive/MyDrive/rus_phraseology/subtitles_processed/txt'
output_csv_path = '/content/gdrive/MyDrive/rus_phraseology/subtitles_processed/movie_info.csv'
extract_movie_info_from_txt(txt_folder_path, output_csv_path, append=False)
sort_csv_alphabetically(output_csv_path)

Processing file: mira.txt
Extracted title: mira
Extracted genres: ['adventure', 'drama', 'sci-fi']
Processing file: baba_yaga_spasaet_mir.txt
Extracted title: baba yaga spasaet mir
Extracted genres: ['adventure', 'comedy', 'family']
Processing file: 1812_ulanskaya_ballada.txt
Extracted title: 1812. ulanskaya ballada
Extracted genres: ['adventure', 'drama', 'history']
Processing file: sokrovishcha_partizanskogo.txt
Extracted title: sokrovishcha partizanskogo lesa
Extracted genres: ['adventure', 'family', 'war']
Processing file: upon_the_magic_roads.txt
Extracted title: upon the magic roads
Extracted genres: ['adventure', 'fantasy']
Processing file: silver_skates.txt
Extracted title: silver skates
Extracted genres: ['adventure', 'drama', 'family']
Processing file: gagarin_first_in_space.txt
Extracted title: gagarin: first in space
Extracted genres: ['adventure', 'biography', 'drama', 'history']
Processing file: moy_papa_vozhd.txt
Extracted title: moy papa - vozhd
Extracted genres: ['adve

In [ ]:
movie_info_df = pd.read_csv(output_csv_path)
movie_info_df.head()

,filename,title,genre
0,100_minutes,100 minutes,"drama, history, war"
1,12,12,"crime, drama, thriller, war"
2,1812_ulanskaya_ballada,1812. ulanskaya ballada,"adventure, drama, history"
3,7_dney_7_nochey,"7 dney, 7 nochey","comedy, family"
4,about_love,about love,"drama, romance, thriller"


###  3. Build a movie_subtitles dataframe

In [ ]:
import pandas as pd
import os
import re

In [ ]:
def process_srt_to_sentences(file_path):
    """
    Reads an SRT file and extracts sentences as a dictionary, ensuring one sentence per entry.
    """
    sentences = {}
    sentence_id = 1

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line:  # Skip blank lines
                continue

            # Split line into sentences based on punctuation
            line_sentences = re.split(r'(?<=[.!?]) +', line)
            for sentence in line_sentences:
                if sentence:  # Ignore empty splits
                    sentences[sentence_id] = sentence.strip()
                    sentence_id += 1

    return sentences

def build_dataframe(csv_path, srt_folder):
    """
    Builds a DataFrame from a CSV and corresponding SRT files.
    """
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Initialize the DataFrame structure
    data = {
        'id': [],
        'title': [],
        'genre': [],
        'text': []
    }

    for index, row in df.iterrows():
        title = row['title']
        genre = row['genre']
        filename = row['filename']

        # Build the SRT file path
        srt_file_path = os.path.join(srt_folder, f"{filename}.srt")
        if not os.path.exists(srt_file_path):
            print(f"Missing SRT file: {srt_file_path}")
            continue

        # Process the SRT file
        sentences = process_srt_to_sentences(srt_file_path)

        # Append to data
        data['id'].append(index + 1)  # Generate sequential IDs
        data['title'].append(title)
        data['genre'].append(genre)
        data['text'].append(sentences)

    # Convert data to DataFrame
    return pd.DataFrame(data)

In [ ]:
srt_folder = '/content/gdrive/MyDrive/rus_phraseology/subtitles_processed/srt'
df = build_dataframe(output_csv_path, srt_folder)
output_path = '/content/gdrive/MyDrive/rus_phraseology/subtitles_processed/rus_sub_df.csv'
df.to_csv(output_path, index=False, encoding='utf-8')
print(df.head())

   id                    title                        genre  \
0   1              100 minutes          drama, history, war   
1   2                       12  crime, drama, thriller, war   
2   3  1812. ulanskaya ballada    adventure, drama, history   
3   4         7 dney, 7 nochey               comedy, family   
4   5               about love     drama, romance, thriller   

                                                text  
0  {1: 'ИВАН ДЕНИСОВИЧ по мотивам рассказа АЛЕКСА...  
1  {1: '"ДВЕНАДЦАТЬ" Господа присяжные заседатели...  
2  {1: '[по-французски] Графиня Валевская, я рад,...  
3  {1: '(Настя) -Давным-давно в одной солнечной с...  
4  {1: 'СУБТИТРЫ: Н.ПЕТРУЛЕВИЧ, И.КИРИК, А.СОЛОНИ...  
